In [1]:
# !pip install sqlite3
import sqlite3

conn = sqlite3.connect('Hospitalinfo.db')
cursor_obj = conn.cursor()

In [2]:
# Exercise 1: Connect to your database server and print its version
cursor_obj.execute("select sqlite_version();")
db_version = cursor_obj.fetchone()
print("You are connected to SQLite version: ", db_version)

You are connected to SQLite version:  ('3.38.2',)


In [3]:
import pandas as pd
#retrieve the query results into a pandas dataframe
df_hospital = pd.read_sql_query("select * from hospital;", conn)
df_hospital

,ID,Name,BedCount
0,101,Mayo Clinic,230
1,102,JP Hoplkins,130
2,103,New Amsterdam,200
3,104,Clevland Clinic,30
4,105,Toronto Hopital,180
5,106,Natura,150
6,107,Johnson and Johnson,600


In [4]:
#retrieve the query results into a pandas dataframe
df_doctor = pd.read_sql_query("select * from doctor;", conn)
df_doctor

,DocID,DocName,HospitalID,JoiningDate,Speciality,Exp,salary
0,1,Michael,101,2005-02-10,Pediatric,10,900000.0
1,2,Linda,101,2007-08-08,Gyane,5,850000.0
2,3,William,102,2004-09-11,Cardialogist,11,1000000.0
3,4,Richard,101,2011-09-05,Pediatric,12,950000.0
4,5,Karen,103,2020-09-05,Oncologist,15,1100000.0
5,6,Robert,104,1998-09-04,Gyane,14,940000.0
6,7,Susan,105,1994-06-05,Oncologist,11,870000.0
7,8,Nancy,106,1994-06-05,Cardialogist,28,870000.0
8,9,Nick,107,2019-06-05,Cardialogist,9,770000.0


In [5]:
# Exercise 2: Fetch Hospital and Doctor Information using hospital Id and doctor Id
def get_doctor_detail(doctor_id):
    try:
        select_query = """select * from doctor where DocID = ?"""
        cursor_obj.execute(select_query, (doctor_id,))
        records = cursor_obj.fetchall()
        print("Printing Doctor record")
        for row in records:
            print("DocID:", row[0])
            print("DocName:", row[1])
            print("HospitalID:", row[2])
            print("JoiningDate:", row[3])
            print("Specialty:", row[4])
            print("Exp:", row[5])
            print("Salary:", row[6])
    except (Exception, sqlite3.Error) as error:
        print("Error while getting data", error)

get_doctor_detail(8)

Printing Doctor record
DocID: 8
DocName: Nancy
HospitalID: 106
JoiningDate: 1994-06-05
Specialty: Cardialogist
Exp: 28
Salary: 870000.0


In [6]:
def get_hospital_detail(hospital_id):
    try:
        select_query = """select * from hospital where ID = ?"""
        cursor_obj.execute(select_query, (hospital_id,))
        records = cursor_obj.fetchall()
        print("Printing Hospital record")
        for row in records:
            print("ID:", row[0])
            print("Name:", row[1])
            print("BedCount:", row[2])
            
    except (Exception, sqlite3.Error) as error:
        print("Error while getting data", error)

get_hospital_detail(105)

Printing Hospital record
ID: 105
Name: Toronto Hopital
BedCount: 180


In [7]:
# Exercise 3: Get the list Of doctors as per the given specialty and salary
def get_doctor_list(specialty, salary):
    try:
        select_query = """select * from doctor where Speciality = ? and Salary = ?"""
        cursor_obj.execute(select_query, (specialty, salary))
        records = cursor_obj.fetchall()
        print("Printing Doctor record")
        for row in records:
            print("DocID:", row[0])
            print("DocName:", row[1])
            print("HospitalID:", row[2])
            print("JoiningDate:", row[3])
            print("Speciality:", row[4])
            print("Exp:", row[5])
            print("Salary:", row[6])
    except (Exception, sqlite3.Error) as error:
        print("Error while getting data", error)

get_doctor_list('Pediatric', 900000.0)

Printing Doctor record
DocID: 1
DocName: Michael
HospitalID: 101
JoiningDate: 2005-02-10
Speciality: Pediatric
Exp: 10
Salary: 900000.0


In [8]:
# Exercise 4: Get a list of doctors from a given hospital
def get_hospital_name(hospital_id):
    try:
        select_query = """select * from hospital where ID = ?"""
        cursor_obj.execute(select_query, (hospital_id,))
        record = cursor_obj.fetchone()
        return record[1]
    except (Exception, sqlite3.Error) as error:
        print("Error while getting data", error)

def get_doctor(hospital_id):
    try:
        hospital_name = get_hospital_name(hospital_id)
        select_query = """select * from doctor where HospitalID = ?"""
        cursor_obj.execute(select_query, (hospital_id,))
        records = cursor_obj.fetchall()
        print("Printing Doctor record of", hospital_name)
        for row in records:
            print("DocID:", row[0])
            print("DocName:", row[1])
            print("HospitalID:", row[2])
            print("JoiningDate:", row[3])
            print("Specialty:", row[4])
            print("Exp:", row[5])
            print("Salary:", row[6])
    except (Exception, sqlite3.Error) as error:
        print("Error while getting data", error)

get_doctor(101)

Printing Doctor record of Mayo Clinic
DocID: 1
DocName: Michael
HospitalID: 101
JoiningDate: 2005-02-10
Specialty: Pediatric
Exp: 10
Salary: 900000.0
DocID: 2
DocName: Linda
HospitalID: 101
JoiningDate: 2007-08-08
Specialty: Gyane
Exp: 5
Salary: 850000.0
DocID: 4
DocName: Richard
HospitalID: 101
JoiningDate: 2011-09-05
Specialty: Pediatric
Exp: 12
Salary: 950000.0


In [9]:
# Exercise 5: Update doctor experience in years
import datetime
from dateutil.relativedelta import relativedelta

def update_experience(doctor_id):
    try:
        select_query = """select * from doctor where DocID = ?"""
        cursor_obj.execute(select_query, (doctor_id,))
        record = cursor_obj.fetchone()
        joining_date = record[3]
            
        joining_date_1 = datetime.datetime.strptime(''.join(map(str, joining_date)), '%Y-%m-%d')
        today_date = datetime.datetime.now()
        experience = relativedelta(today_date, joining_date_1).years
        
        sql_select_query = """update doctor set Exp = ? where DocId = ?"""
        cursor_obj.execute(sql_select_query, (experience, doctor_id))
        conn.commit()
        print("Doctor Id:", doctor_id, " Experience updated to ", experience, " years")
        
    except (Exception, sqlite3.Error) as error:
        print("Error while getting data", error)

update_experience(8)

Doctor Id: 8  Experience updated to  28  years
